 # <center> WalsonScript 设计文档

## 1. 设计内容
本次设计基于JavaScript语言，设计了一套英文编程语言WalsonScript，能实现词法分析、语法分析、语法制导翻译。（1）根据LL(1)等文法规则的需要，用栈的思想定义一种数据结构——流，将字符流转成符号流。（2）在词法分析部分实现了分词断句，能判断词性，能处理var、int、float、string、boolean五种数据类型和”+\-*/><=!&|^%”几种基本运算符。（3）在语法分析部分针对不同的语法构建抽象语法树，能处理以下几种语义：表达式语义Expression、单值成分语义Factor（变量语义Variable、数值语义Scalar）、声明语义Stmt（赋值语义AssignState、代码块语义Block、声明语义DeclareState、循环语义ForState、条件分支语义IfState、函数声明语义FuncState、函数变量语义FuncArgs、函数调用语义CallExpr、返回语义ReturnState）。（4）使用三地址代码作为中间代码表示，在语法制导翻译部分定义符号类型：常数、变量、标签，使用符号表和静态符号表存储符号，自顶向下翻译程序。（5）最终可以编写虚拟机运行代码。

WalsonScript的主要贡献体现在：（1）作为脚本语言，能满足常规运算需求编写基本代码进行基本运算，比JavaScript书写更便捷，无冗余的复杂功能和语法规则，并且句末无需追加分号；代码风格和数字表示类型接近Python语言，但是无需考虑缩进和代码内部空格、换行的格式问题。定义了五种数据类型，在进行代码编写时避免了Python语言会遇到的接收数据类型不清晰、难以分类处理的问题。在保留字的定义上兼容性更强，比如对Boolean类型的定义，大小写字母均合法。（2）WalsonScript在底层使用流的数据结构获取、处理和存储输入的数据，规定了数字类型变量的合法区间（定义域属于区间[-215, 215-1]）和变量名的合法长度（长度不超过25）。使用Enum类存储节点的类型，进行后续的匹配处理，实则是比较对象的地址。以上这些设计使得WalsonScript进行计算时效率得到优化，能满足绝大部分基本运算的效率需求。（3）在词法分析和翻译时，针对不同的异常情况和代码非法编写进行报错提示。（4）能够使用虚拟机进行代码运行。

下文将按照编译的流程对各个部分的设计原理和程序进行介绍。

## 2 词法分析器（Lexer）
### 2.1 主要功能
词法分析器将字符流转成符号流。每个符号（词法单元，Token）是一个元组tuple，至少包括一个字符串和一个词性描述（符号类型），例如（126，INTEGER）。词法分析的任务即为根据词法类型，识别并读入每个词法单元，将各个词法单元组成一个流结构，以符号流的结构输出。

### 2.2 设计
#### 2.2.1 流
流（Stream）是一种随着时间推移逐渐产生的可用数据序列。一般情况下，流需要提供获取下一个数据的接口（next、hasNext方法）。next方法读取到一个数据后，这个数据就相当于已经流过，因此无法重复读取，也就表明流的每一个处理节点所需的内存空间很小。

#### 2.2.2 词法映射
词法分析器读入字符流，按照词法单元属性，做词法映射到保留字（关键字）KEYWORD、变量名VARIABLE、运算符OPERATOR、分解符BRACKET、数据类型INTEGER、FLOAT、BOOLEAN、STRING等，生成符号（词法单元）流，即按照每个词法单元的类型存入整条字符流，在后续使用时能输出词法单元流，流中是有值和具体类型的词法单元。
其中，KEYWORD包括("var", "if", "else", "for", "while", "break", "func", "return", "int", "float", "bool", "void", "string")。但是并非所有串都是语言支持的，通常用一些约束规则来描述串，正则表达式就是其中之一。对于VARIABLE，对应的正则表达式为`/^[_a-zA-Z0-9]$/`，包含下划线、大小写字母、数字。OPERATOR对应的正则表达式为`/^[+\-*/><=!&|^%,]$/`。其余几种映射因为在代码和逻辑中特殊的结构，有特别的判别方式，不直接使用正则表达式判断，比如BRACKET类可以根据“(){}”符号直接匹配判断，STRING类型可以根据引号“"/’”识别读取，INTEGER和FLOAT类可以根据小数点“.”和数字的自动机判别，BOOLEAN类可以直接判断是否包含“true/TRUE”“false/FALSE”。

### 2.3 词法分析器整体程序
![jupyter](./image/1.png)

### 2.4 各部分自动机设计
编译器制作过程中一般使用正则表达式来表述词法，然后用状态机来实现正则表达式。在本次设计的词法分析器中，所包含的自动机如下：

#### 2.4.1 词法映射：KEYWORDS VARIABLE
对应函数：selectVarKey
功能：区分关键字和变量名。
自动机：
![jupyter](./image/2.png)

处理流程：
- 初始状态0状态读取，如果属于Literal类正则表达式，则转入状态1；
- 状态1继续读入内容，如果是Literal类正则表达式，则转换为状态2，如果是符号，则转入token状态；
    - 在状态2中，可以自循环，直到读取内容属于非上述类型的正则表达式（可能是符号），则转入下一个状态；
    - 在token状态中，如果在字典中，则是关键字，否则为变量。

测试用例：if a
State 0：read ‘i’  ->  State 1: read ’f’  ->  State 2: read ‘ ’  ->  State Token: determine, belongs to the set  ->  KEYWORDS

State 0：read ‘a’  ->  State 1: read ’ ’  ->  State Token: determine, not belongs to the set  ->  VARIABLE

#### 2.4.2 词法映射：STRING
对应函数：makeString
功能：读入不同引号表达的字符串。
自动机：
![jupyter](./image/3.png)

处理流程：
- 初始状态0开始读取，读入““”则进入状态1；读入“‘”则进入状态2；
- 状态1读入除“””之外的符号，则自循环；若读入“””，则将其加到现有的Token末尾，返回整个字符串，结束读入；
- 状态2与状态1原理相同。

测试用例：‘a1_’
State 0: read “‘”  ->  State 2: read ‘a’  ->  State 2: read ‘1’  ->  State 2: read ‘_’  ->  State 2: read “’”  ->  return String

#### 2.4.3 词法映射：OPERATOR
对应函数：makeOperator
功能：读入不同的操作符，进行运算判断。
自动机：
![jupyter](./image/4.png)

处理流程：
初始状态0开始读取，12种运算符分为两种情况：
（1）+-><&|^：可以自己重复（++）；可以和=复合使用（+=）；可以单独使用（+）；
（2）*/=!%：可以和=复合使用（*=）；可以单独使用（*）。

#### 2.4.4 词法映射：NUMBER
对应函数：makeNumber
功能：读入有符号数和无符号数，进行数字的格式合格性校验。
自动机：

处理流程：
初始状态0开始读取：
- 读入0则转入状态1（处理0开头的数字串）：
    - 循环读入0则保持状态1；
    - 读入“.”则转入状态4；
    - 读入1-9则转入状态2；
    - 否则返回INTEGER类的字符串数字，即全0串。
- 读入1-9则转入状态2（处理含有整数部分的数字串）：
    - 循环读入1-9则保持状态2；
    - 读入“.”则转入状态4；
    - 否则返回INTEGER类的字符串数字，即整数串（串前可包含0）。
- 读入“+”“-”则转入状态3（处理有符号数字）：
    - 读入0-9则转入状态2；
    - 读入“.”则转入状态5；
    - 否则返回Error，即仅有符号没有数字的情况。
- 读入“.”则转入状态5（处理不含整数部分的小数）：
    - 读入0-9则转入状态4；
    - 否则返回Error，即仅有小数点没有数字的情况。
- 状态4（处理含有整数部分的小数）：
    - 循环读入0-9则保持状态4；
    - 读入“.”则返回Error，即两个小数点连用的情况；
    - 否则返回FLOAT类的字符串数字，即含整数部分的小数串（串前可包含0）。

从上述处理流程可以看出，WalsonScript允许几种数字表示类型：
- 全0，00…00
- 整数串，12…34
- 小数串，12…34.12…34
- 不含整数部分的小数串，.12…34
- 以0开头的（1）（2）（3）的情况
- 有符号数，以“+”或“-”开头的上述五种类型

### 2.5 代码部分
#### 2.5.1 lexer文件
CharType：

存储字母、数字、变量名、操作数四种类型的正则表达式和判断函数。

Lexer：

词法分析器，通过词法单元信息处理空格、换行符、注释、分隔符、字符串、关键字、变量名、有符号数、无符号数、操作符几种词法类型。

In [1]:
 analyse(source) {
        const tokens = [];
        const it = new PeekIterator(source, "\0");

        while (it.hasNext()) {
            let c = it.next();
            if (c == "\0") {
              break;
        }
        let lookahead = it.peek();

        // Do not process whitespace, '\n', and '\r'.
        if (c == " " || c == "\n" || c == "\r") {
            continue;
        }

        // Annotation
        if (c == "/") {
            // If the prefix is '//', then omit all the following contents until reaching the tail '\n'.
            if (lookahead == "/") {
              while (it.hasNext() && (c = it.next()) != "\n");
              continue;
            }
            // If the prefix is '/*', then omit all the following contents until reaching the '*/'.
            else if (lookahead == "*") {
                let flag = false;
                while (it.hasNext()) {
                  const p = it.next();
                  if (p == "*" && it.peek() == "/") {
                    flag = true;
                    it.next();
                    break;
                  }
              }

            if (!flag) {
                throw new LexerException("comment not matched");
            }
            continue;
        }
      }

        // Omit brackets.
        if (c == "{" || c == "}" || c == "(" || c == ")") {
          tokens.push(new Token(TokenType.BRACKET, c));
          continue;
        }

        // Read a string.
        if (c == '"' || c == "'") {
          it.putPeek();
          tokens.push(Token.makeString(it));
          continue;
        }

        // Split keywords and variables.
        if (CharType.isLetter(c)) {
          it.putPeek();
          tokens.push(Token.selectVarKey(it));
          continue;
        }

        // Read unsigned numbers.
        if (CharType.isNumber(c)) {
          it.putPeek();
          tokens.push(Token.makeNumber(it));
          continue;
        }

        // Read numbers with a sign.
        if ((c == "+" || c == "-") && CharType.isNumber(lookahead)) {
          
          // Omit
          // a+1 (lastToken == 'a'  char == '+'  CharType.isNumber(lookahead))
          // 1+1 (lastToken == '1'  char == '+'  CharType.isNumber(lookahead))
          // Read
          // +1 (lastToken == 'null'  char == '+'  CharType.isNumber(lookahead))
          // 1*-1 (lastToken == '*'  char == '-'  CharType.isNumber(lookahead))
          const lastToken = tokens[tokens.length - 1] || null;

          if (lastToken == null || !lastToken.isValue()) {
            it.putPeek();
            tokens.push(Token.makeNumber(it));
            continue;
          }
        }

        // Read operators.
        if (CharType.isOperator(c)) {
          it.putPeek();
          tokens.push(Token.makeOperator(it));
          continue;
        }

        throw LexerException.fromChar(c);
      }
      return tokens;
}


SyntaxError: invalid syntax (2039944995.py, line 1)

LexerException：

词法分析的报错函数，若出现异常则返回异常的字符，打印出`Unexpected character ${char}`异常信息。

Token：

词法单元函数，储存关键字词典，以及数据类型、数值等判断函数，定义变量和关键字、字符串、操作数、数字四种词法单元的自动机处理机制（词法单元构造函数）。

In [ ]:
const Keywords = new Set([
      "var",
      "if",
      "else",
      "for",
      "while",
      "break",
      "func",
      "return",
      "int",
      "float",
      "bool",
      "void",
      "string"
    ]);
    
    class Token {
      // if a = 1
      static selectVarKey(it) {
        let str = "";
    
        // Loop while it has next value.
        // Peek each value and concatenate the literal one to the string.
        while (it.hasNext()) {
            const char = it.peek();
    
            if (CharType.isLiteral(char)) {
              str += char;
            } 
            else {
                break;
            }
            // Logically equal to i++, it is the alleged "iterator"!
            it.next();
    }
    
        // Judge specific data type.
        if (Keywords.has(str)) {
            return new Token(TokenType.KEYWORD, str);
    }
    
        // Ignore the type of Boolean case, both lowercase and uppercase are valid.
        if (str == "true" || str == "false" || str == "TRUE" || str == "FALSE") {
          return new Token(TokenType.BOOLEAN, str);
    }
    
        return new Token(TokenType.VARIABLE, str);
      }
    
      // "str...", 'str...'
      static makeString(it) {
    let str = "";
    
    let state = 0;
    
        while (it.hasNext()) {
            // The difference from the former selectCharType:
            // This is a Finite State Machine, and the 0-1 state, to be exact,
            // if the value is invalid, we do not have to peek it.
            let char = it.next();
    
            switch (state) {
              case 0:
                if (char == '"') {
                  state = 1;
                }
                else {
                  state = 2;
                }
                str += char;
                break;
              case 1:
                if (char == '"') {
                  return new Token(TokenType.STRING, str + char);
                }
                else {
                  str += char;
                }
                break;
              case 2:
                if (char == "'") {
                  return new Token(TokenType.STRING, str + char);
                }
                else {
                  str += char;
                }
                break;
            }
        }
        throw new LexerException("Unexpected Error in makeString!");
      }
    
      // +, ++, +=
      static makeOperator(it) {
        let state = 0;
        while (it.hasNext()) {
            let lookahead = it.next();
    
            switch (state) {
              case 0:
                switch (lookahead) {
                  case "+":
                    state = 1;
                    break;
                  case "-":
                    state = 2;
                    break;
                  case "*":
                    state = 3;
                    break;
                  case "/":
                    state = 4;
                    break;
                  case ">":
                    state = 5;
                    break;
                  case "<":
                    state = 6;
                    break;
                  case "=":
                    state = 7;
                    break;
                  case "!":
                    state = 8;
                    break;
                  case "&":
                    state = 9;
                    break;
                  case "|":
                    state = 10;
                    break;
                  case "^":
                    state = 11;
                    break;
                  case "%":
                    state = 12;
                    break;
                  case ",":
                    return new Token(TokenType.OPERATOR, ",");
                  case ";":
                    return new Token(TokenType.OPERATOR, ";");
                }
                break;
              case 1: {
                if (lookahead == "+") {
                  return new Token(TokenType.OPERATOR, "++");
                }
                else if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "+=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "+");
                }
              }
              case 2:
                if (lookahead == "-") {
                  return new Token(TokenType.OPERATOR, "--");
                }
                else if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "-=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "-");
                }
              case 3:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "*=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "*");
                }
              case 4:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "/=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "/");
                }
              case 5:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, ">=");
                }
                else if (lookahead == ">") {
                  return new Token(TokenType.OPERATOR, ">>");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, ">");
                }
              case 6:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "<=");
                }
                else if (lookahead == "<") {
                  return new Token(TokenType.OPERATOR, "<<");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "<");
                }
              case 7:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "==");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "=");
                }
              case 8:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "!=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "!");
                }
              case 9:
                if (lookahead == "&") {
                  return new Token(TokenType.OPERATOR, "&&");
                }
                else if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "&=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "&");
                }
              case 10:
                if (lookahead == "|") {
                  return new Token(TokenType.OPERATOR, "||");
                }
                else if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "|=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "|");
                }
              case 11:
                if (lookahead == "^") {
                  return new Token(TokenType.OPERATOR, "^^");
                }
                else if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "^=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "^");
                }
              case 12:
                if (lookahead == "=") {
                  return new Token(TokenType.OPERATOR, "%=");
                }
                else {
                  it.putPeek();
                  return new Token(TokenType.OPERATOR, "%");
                }
            }
    }
    
        throw new LexerException("Unexpected error in makeOperator!");
      }
    
      // 123, 1.23, .23, 0, 0123, 0.23, -123
      static makeNumber(it) {
        let state = 0;
    let str = "";
    
        while (it.hasNext()) {
          let lookahead = it.peek();
    
          switch (state) {
            case 0:
              if (lookahead == "0") {
                state = 1;
              }
              else if (CharType.isNumber(lookahead)) {
                state = 2;
              }
              else if (lookahead == "+" || lookahead == "-") {
                state = 3;
              }
              else if (lookahead == ".") {
                state = 5;
              }
              break;
            case 1:
              if (lookahead == "0") {
                state = 1;
              }
              else if (lookahead == ".") {
                state = 4;
              }
              else if (CharType.isNumber(lookahead)) {
                state = 2;
              }
              else {
                return new Token(TokenType.INTEGER, str);
              }
              break;
            case 2:
              if (CharType.isNumber(lookahead)) {
                state = 2;
              }
              else if (lookahead == ".") {
                state = 4;
              }
              else {
                return new Token(TokenType.INTEGER, str);
              }
              break;
            case 3:
              if (CharType.isNumber(lookahead)) {
                state = 2;
              }
              else if (lookahead == ".") {
                state = 5;
              }
              else {
                throw LexerException.fromChar(lookahead);
              }
              break;
            case 4:
              if (lookahead == ".") {
                throw LexerException.fromChar(lookahead);
              }
              else if (CharType.isNumber(lookahead)) {
                state = 4;
              }
              else {
                return new Token(TokenType.FLOAT, str);
              }
              break;
            case 5:
              if (CharType.isNumber(lookahead)) {
                state = 4;
              }
              else {
                throw LexerException.fromChar(lookahead);
              }
              break;
          }
          str += lookahead;
          it.next();
        }
        throw new LexerException("Unexpected error in makeNumber!");
      }
    }
    

TokenType：

存储保留字（关键字）KEYWORD、变量名VARIABLE、运算符OPERATOR、分解符BRACKET、数据类型INTEGER、FLOAT、BOOLEAN、STRING的Enum类信息。

#### 2.5.2 common文件
arrayGenerator：

数组生成器，用于测试用例中，将输入的string类型的代码转换为数组类型，便于获取每一个字符。

Enum：

因为JavaScript没有枚举类，若使用int型，则后续需要转为字符串；若使用string型，则需比较每个字符，时间开销太大。使用Enum则是比较对象，实际为比较地址，即内存中的数字，从底层结构上提高了WalsonScript的代码处理和运行性能。

PeekIterator：

流结构迭代器的相关功能。包含next、hasNext、peek、putPeek四个功能函数，实现下一个词法单元的查看和跳过等功能。

#### 2.5.3 tests文件
CharType.test：

测试单个词法单元类型判断函数。
![jupyter](./image/6.png)

PeekIterator.test：

测试流结构迭代器的相关功能。包含next、hasNext、peek、putPeek四个功能函数，实现下一个词法单元的查看和跳过等功能。
![jupyter](./image/7.png)

Token.test：

测试四个用于主要词法单元类型处理的自动机函数。
![jupyter](./image/8.png)


Lexer.test：

测试一行代码、一个函数、一个目标代码文件的词法分析效果。
![jupyter](./image/9.png)
![jupyter](./image/10.png)
![jupyter](./image/11.png)
![jupyter](./image/12.png)



## 3 语法分析器
### 3.1 主要功能
语法分析器读取词法分析器处理过的词法单元流，通过对英文输入的代码语法、语义识别（包含表达式语义Expression、单值成分语义Factor（变量语义Variable、数值语义Scalar）、声明语义Stmt（赋值语义AssignState、代码块语义Block、声明语义DeclareState、循环语义ForState、条件分支语义IfState、函数声明语义FuncState、函数变量语义FuncArgs、调用语义CallExpr、返回语义ReturnState）），按照每个词法单元对应的语义类型存入抽象语义树（后序遍历），然后可以从抽象语法树中读取每个代码对应的各语义部分。

### 3.2 设计
#### 3.2.1 抽象语法树（Abstract Syntax Tree）
使用抽象语法树进行语法分析能够隐藏细节，与思考无关的细节。例如在Python程序中无需使用分号分隔语句；不需要花括号表示代码块，直接使用缩进；if语句后也可以不使用括号。

本次设计中程序部分对应的抽象语法树如下图所示。在这一树中，每个节点代表源代码中的一种结构（一个.js文件），携带关键信息。
![jupyter](./image/13.png)

#### 3.2.2 语法分析器（Parser）
根据语法规则，将符号流（lexeme, token）转化为抽象语法树。编译器只能识别上下文无关文法（Content-Free Grammar），即推导符左边只有一个非终结符，推导符右边的部分长度不小于左边的文法。所以，需要将文法转换为上下文无关文法进行处理。

#### 3.2.3 语法规则
一般用产生式描述语法规则，例如：

$$Expression ::= Expression + 1 \mid 1$$

其中，Expression是非终结符，1是终结符，对应词法单元。

非终结符（递归）函数，生成一个非叶子节点。终结符函数，生成一个叶子节点。但是在递归函数中，需要考虑左递归造成的死循环问题，如：

$$A ::= A\alpha \mid \beta$$

根据左递归文法生成的文法特点，我们可以使用两个等效的产生式替代原本文法，即用右递归替代左递归：

$$A ::= \beta A^{'}$$
$$A^{'} ::= \alpha A^{'} \mid \epsilon$$

更复杂地，若左递归为：

$$A ::= A\alpha \mid A\beta \mid A\gamma \mid \rho$$

可以将其替代为：

$$A ::= \rho A^{'}$$
$$A ::= \alpha A^{'} \mid \beta A^{'} \mid \gamma A^{'} \mid \epsilon$$


#### 3.2.4 操作符优先级
WalsonScript中在语法分析器部分还定义了操作符的优先级，自上而下递增为：
- &  |  ^
- ==  !=  >  <  >=  <=
- +  -
- /  *
- <<  >>

### 3.3 语法分析器整体程序
![jupyter](./image/14.png)

### 3.4 底层各部分函数逻辑：

#### 3.4.1 赋值语义：AssignState
![jupyter](./image/15.png)

#### 3.4.2 声明语义：DeclareState
![jupyter](./image/16.png)

#### 3.4.3 函数声明语义：FuncDeclare
![jupyter](./image/17.png)

#### 3.4.4 返回语义：ReturnState
![jupyter](./image/18.png)

#### 3.4.5 条件分支语义：IfState
![jupyter](./image/19.png)
![jupyter](./image/20.png)

#### 3.4.6 表达式语义：Expression
![jupyter](./image/21.png)

#### 3.4.7 代码块语义：Block
![jupyter](./image/22.png)

### 3.5 代码部分：
#### 3.5.1 parser/ast文件
ASTreeNode：

定义抽象语法树的基本结构和函数。

NodeTypes：

用Enum类存储各个底层语义函数的类型名称信息。

In [ ]:
module.exports = {
        BLOCK : new Enum("BLOCK", 1),
        BINARY_EXPR: new Enum("BINARY_EXPR", 2),
        UNARY_EXPR: new Enum("UNARY_EXPR", 3),
        VARIABLE: new Enum("VARIABLE", 4),
        IF_STMT: new Enum("IF_STMT", 5),
        WHILE_STMT: new Enum("WHILE_STMT", 6),
        FOR_STMT: new Enum("FOR_STMT", 7),
        ASSIGN_STATE : new Enum("ASSIGN_STATE", 8),
        FUNCTION_DECLARE_STMT: new Enum("FUNCTION_DECLARE_STMT", 9),
        DECLARE_STATE : new Enum("DECLARE_STATE", 10),
        SCALAR : new Enum("SCALAR", 11),
        RETURN_STMT :new Enum("RETURN_STMT", 12),
        FUNCTION_ARGS :new Enum("FUNCTION_ARGS", 13),
        CALL_EXPR : new Enum("CALL_EXPR", 14)
    }

index：

存储各个底层语义函数的导出信息，用于解决在代码文件中直接导入其他文件的循环调用的冲突问题。 

其余文件均用于实现底层语义函数，逻辑流程如上一小节中的图示。

#### 3.5.2 parser/util文件
ParseException：

语法分析的报错函数，若出现异常则返回异常的词法单元，打印出`Syntax Error, unexpected token ${token.getValue()}`异常信息。

ParserUtil：

定义后续遍历和广度优先两种算法。

PeekTokenIterator：

继承PeekIterator类，定义了nextMatch和nextMatchType两个函数，用于跳过底层语义函数固定结构形式中的一些词法单元，比如赋值语义中的“=”，返回语义中的“return”，函数声明语义中的“int”等。

PriorityTable：

按优先级顺序导出存储的操作符。

#### 3.5.3 parse文件
Parser：

定义语义分析函数，首先对读取的字符串进行词法分析，然后调用Program.parse函数，自顶向下进行语义分析。定义文件读取解析函数。

SimpleParser：

用于简单测试的右递归累加文法。

#### 3.5.4 tests文件
ParseExpression.test：

将一行代码按后序遍历读取。
![jupyter](./image/23.png)

SimpleParser.test：

测试简单的右递归累加文法。
![jupyter](./image/23.png)
![jupyter](./image/24.png)
![jupyter](./image/25.png)

Stmts.test：

测试不同语义的代码被语义函数处理后的形式。
![jupyter](./image/26.png)
![jupyter](./image/27.png)
![jupyter](./image/28.png)

## 4 中间代码生成
使用三地址代码生成中间代码。三地址代码是一行最多只有三个地址的代码。例如有代码：

In [ ]:
var a = 1 * 2 + 3 * 4

可以使用三地址代码表示为：

In [ ]:
p0 = 1 * 2
p1 = 3 * 4
a = p0 + p1

将上述三个步骤映射到CPU指令会比较容易，在实现翻译时形成结构化思维、组装复用，将目标代码拆解成中间代码。比如：

If Statement程序：

In [ ]:
if (a + b > 1){
    c = 2
    }
    else {
    c = 3
    }

其三地址代码表示为：

In [ ]:
p0 = a + b
p1 = p0 > 1
IF p1 ELSE L0:
c = 2
GOTO L1
L0: 
c = 3
L1:

For Statement程序:

In [ ]:
for (var i = 0; i < 10; i ++){
    sum += i
    }

其三地址代码表示为：

In [ ]:
i = 0
L0: p0 = i < 10
If p0 ELSE L1
sum = sum + i
i = i + 1
p0 = i < 10
GOTO L0
L1:

Func Statement程序:

In [ ]:
func sample(a, b, c){
    return true
    } 
sample(1, 2, 3)

其三地址代码表示为：

In [ ]:
SAMPLE:
// Statements...
RETURN true

PASS 1 
PASS 2
PASS 3
CALL SAMPLE

三地址代码的程序架构逻辑如下图所示。一个三地址代码程序会有多个三地址代码指令，三地址代码指令的属性对应有type（指令类型，如IF、ASSIGN）、result（三地址代码在等号左边的部分表示结果，为地址类型的符号）、operator（三地址代码在等号右边部分的操作符，如+-*/）、arg（三地址代码在等号右边部分的两个参数，可能是常数、变量或标签，即下面一节中将要介绍的符号）：

![jupyter](./image/29.png)

三地址代码的指令类型包括：ASSIGN、GOTO、IF、LABEL、CALL、RETURN、SP（Stack Pointer，栈指针）、PARAM（Parameter，参数）。

## 5 语法制导翻译
语法制导定义（Syntax Directed Definition）定义抽象语法树如何被翻译。将文法分为两部分：属性（用于存储中间值和结果）和规则（描述属性如何被计算）。整体的翻译程序是一个自顶向下的设计，输入为抽象语法树：
![jupyter](./image/30.png)

### 5.1 符号抽象
将在WalsonScript的符号表和静态符号表等存储空间中存储的元素称为符号。将符号进行抽象分类，符号可以是常量、变量（有地址的符号）、标签（如GOTO等跳转语句的符号）。
![jupyter](./image/31.png)

### 5.2 词法作用域
词法作用域指翻译时一个符号的可见范围，是一种中间的记录来描述符号之间的关系，比如给程序中的一个变量a重新赋值时，两个变量a的关系。词法作用域和源代码的书写相关，并在运行时生效。

翻译时作用域的处理过程。例，有代码：

In [ ]:
var a = 1 * 2 + 3
var b = a + 1

对应的三地址代码：

In [ ]:
p0 = 1 * 2
p1 = p0 + 3
a = p1
b = a + 1

对应符号表：

In [ ]:
p0
p1
a
b

映射到运行时：

In [ ]:
SP + 0
SP + 4
SP + 8
SP + 12

### 5.3 符号表（Symbol Table）
符号表是用于存储符号（变量、常量、标签）在源代码中的位置、数据类型，位置信息决定的词法作用域和运行时的相对内存地址。一般用哈希表加树的数据结构实现。
一个符号表包含符号列表部分和符号的所有子节点部分，子节点的数据结构也是符号表。比如有下列代码：

In [ ]:
var a = 0
var b = 1
{
c = b + 1
d = c + 1
}
{
var e = 0
var f = 1
}

 其符号表的数据结构可以表示为：
 
![jupyter](./image/32.png)

 ### 5.4 静态符号表（Statistic Symbol Table）
 静态符号表用于存储常量在常量区域中的位置。一般用哈希表实现。
 
 ### 5.5 代码部分
 #### 5.5.1 translator/symbol文件
 SymbolType：

 用Enum类存储表示符号抽象的三种类型：常量、变量和标签。
 
 Symbol：

 定义符号抽象三种类型的生成函数、拷贝函数、读写访问器函数。
 
 SymbolTable：

 定义符号表类，以及各类符号表函数：往符号表中添加一个符号、增加子符号表、符号表读写访问器、标签创建函数、变量创建函数等。
 
 StatisticSymbolTable：
 
 定义静态符号表。

In [ ]:
class StaticSymbolTable {
        constructor(){
            this.symbols = []
            this.offsetMap = new Map() 
            this.offsetCounter = 0
        }
    
        add(symbol){
            var lexval = symbol.getLexeme().getValue()
            if(!this.offsetMap.has(lexval)) {
                this.offsetMap.set(lexval, symbol)
                symbol.setOffset(this.offsetCounter++)
                this.symbols.push(symbol)
            } else {
                var sameSymbol = this.offsetMap.get(lexval)
                symbol.setOffset(sameSymbol.offset)
            }
    }
    
        size(){
            return this.symbols.size()
        }
    }
    
    module.exports = StaticSymbolTable

### 5.5.2 translator文件
TAInstructionType：

存放三地址代码的指令类型：ASSIGN、GOTO、IF、LABEL、CALL、RETURN、SP、PARAM。

TAInstruction：

定义三地址代码的指令构造函数、三地址代码指令属性的读写访问器函数、指令生成函数。

In [ ]:
constructor(type, result, op, arg1, arg2) {
        this.op = op
        this.type = type
        this.arg1 = arg1
        this.arg2 = arg2
        this.result = result
        this.label = null
      }
    
      toString() {
        switch (this.type) {
          case TAInstructionType.ASSIGN:
            if (this.arg2 != null) {
              return `${this.result} = ${this.arg1} ${this.op} ${this.arg2}`
            } else {
              return `${this.result} = ${this.arg1}`
            }
          case TAInstructionType.IF:
            return `IF ${this.arg1} ELSE ${this.arg2}`
          case TAInstructionType.GOTO:
            return `GOTO ${this.arg1}`
          case TAInstructionType.LABEL:
            return `${this.arg1}:`
          case TAInstructionType.RETURN:
            return `RETURN ${this.arg1}`
          case TAInstructionType.PARAM:
            return `PARAM ${this.arg1} ${this.arg2}`
          case TAInstructionType.SP:
            return `SP ${this.arg1}`;
          case TAInstructionType.CALL:
            return `CALL ${this.arg1}`
        }
        throw new Error("Unkonw opcode type:" + this.type);
      }    

TAProgram：

定义三地址代码程序的构造函数、三地址代码程序的转字符串函数、标签更新函数、静态符号表设置函数。通过TAProgram处理一段程序，不断添加指令，遍历三地址代码中的符号，生成静态符号表。

Translator：

自顶向下的语法制导翻译完整程序。处理逻辑如第五节的图所示，下面展示主要代码：

In [ ]:
translate(astNode) {
        // Tree Address, TA
        const program = new TAProgram()
        const symbolTable = new SymbolTable()
    
        for (const child of astNode.getChildren()) {
          this.translateStmt(program, child, symbolTable)
    }
    
        program.setStaticSymbols(symbolTable)
        return program
      }
    
    
translateStmt(program, node, symbolTable) {
        switch (node.getType()) {
          case NodeTypes.ASSIGN_STATE:
            this.translateAssignState(program, node, symbolTable);
            return;
    
          case NodeTypes.DECLARE_STATE:
            this.translateDeclareState(program, node, symbolTable);
            return;
    
          case NodeTypes.BLOCK:
            this.translateBlock(program, node, symbolTable);
            return;
    
          case NodeTypes.IF_STMT:
            this.translateIfState(program, node, symbolTable);
            return;
    
          case NodeTypes.FUNCTION_DECLARE_STMT:
            this.translateFunctionDeclareState(program, node, symbolTable);
            return;
    
          case NodeTypes.RETURN_STMT:
            this.translateReturnState(program, node, symbolTable);
            return;
    
          case NodeTypes.CALL_EXPR:
            this.translateCallExpr(program, node, symbolTable);
            return;
        }
        throw new Error("Translator not impl. for " + node.getType())
      }
    
    
translateExpr(program, node, symbolTable) {
        if (node.isValueType()) {
          const addr = symbolTable.createSymbolByLexeme(node.getLexeme())
          node.setProp("addr", addr)
          return addr
        }
        else if (node.getType() == NodeTypes.CALL_EXPR) {
          const addr = this.translateCallExpr(program, node, symbolTable)
          node.setProp("addr", addr)
          return addr
        }
        else if (node instanceof Expression) {
          for (const child of node.getChildren()) {
            this.translateExpr(program, child, symbolTable)
          }
          if (node.getProp("addr") == null) {
            node.setProp("addr", symbolTable.createVariable())
          }
          const instruction = new TAInstruction(
            TAInstructionType.ASSIGN,
            node.getProp("addr"),
            node.getLexeme().getValue(),
            node.getChild(0).getProp("addr"),
            node.getChild(1).getProp("addr")
          )
          program.add(instruction)
          return instruction.getResult()
        }
        throw new Error("Unexpected node type :" + node.getType());
      }
    

### 5.5.3 tests文件
Symbol.test：

测试符号表中符号的创建、当前符号表中栈的大小情况、符号表父子之间的层次依赖关系和符号表偏移量的变化。
![jupyter](./image/33.png)
![jupyter](./image/34.png)
![jupyter](./image/35.png)

TransExpr.test：

测试表达式的翻译，读入字符串，进行对Program的自顶向下分析，然后对表达式语法制导翻译，得到该段字符串代码对应的三地址代码。
![jupyter](./image/36.png)
![jupyter](./image/37.png)
![jupyter](./image/38.png)

TransBlock.test：

测试语句块的翻译，读入代码，进行对Program的自顶向下分析，然后语法制导翻译，得到该段代码对应的三地址代码。SP -1表示进入Block后对父节点部分的压栈操作，SP 1表示跳出Block后对之前父节点部分栈存储情况的恢复。SP的变化也可以理解为栈指针的移动，由于栈是先入后出、自下而上存储的，下方地址比上方高，所以父级Block中先声明的变量地址高于子级Block中后续变量的地址，进入子级Block栈指针需要做减法，跳出子级Block栈指针需要做加法。
![jupyter](./image/39.png)


在Block外声明三个变量，再次观测处理Block时栈指针的变化情况。
![jupyter](./image/40.png)


语法制导翻译时的报错处理，在声明时反复声明变量则会出现预设的"Syntax Error, Lexeme " + lexeme.getValue() + " is already defined."报错信息提示。
![jupyter](./image/41.png)


根据报错处理修改代码，将Block中的声明语句改为赋值语句，对变量b重新赋值，代码成功翻译。 
![jupyter](./image/42.png)


TransIfStmt.test：

测试If、If-Else、If-Else if-Else语句代码的翻译，读入代码（If-Else if-Else代码从外部文件通过文件流读入），进行对Program的自顶向下分析，然后语法制导翻译，得到该段代码对应的三地址代码。
![jupyter](./image/43.png)
![jupyter](./image/44.png)
![jupyter](./image/45.png)
![jupyter](./image/46.png)
![jupyter](./image/47.png)

TransFunc.test：

测试两数相乘函数和阶乘递归函数的代码翻译，读入代码（If-Else if-Else代码从外部文件通过文件流读入），进行对Program的自顶向下分析，然后语法制导翻译，得到该段代码对应的三地址代码。
![jupyter](./image/48.png)
![jupyter](./image/49.png)
![jupyter](./image/50.png)
![jupyter](./image/51.png)

## 6 高级功能 虚拟机（Virtual Machine） 
虚拟机是模拟计算机的软件，使用虚拟机可以跨平台操作，屏蔽操作系统底层的不一致，也更适合资源调度共享，在用户使用时互不相干。WalsonScript虚拟机的内存使用32位整数数组表示，寄存器（包括特殊寄存器PC、SP、RA和普通寄存器S0、S1、S2）使用32位整数数组，CPU循环流程服从:

$$fetch \rightarrow decode \rightarrow execute \rightarrow store \rightarrow fetch$$

虚拟机工作流程如下：
![jupyter](./image/52.png)

在生成了三地址代码之后，使用代码生成器将其转换为操作码，即虚拟机上执行的虚拟机器代码。如果直接用虚拟机执行三地址代码，仍然需要考虑例如p1 = p0 + a中运算符号处理的问题，选择匹配运算方法仍然存在时间开销，会影响性能；对p0和a的存储也涉及到寄存器的选择、利用问题。

WalsonScript的代码生成器类型设计如下图所示。代码生成器生成的一个程序（Program）包含多条指令（Instruction），每条指令对应一个操作码（OpCode）和多个操作数（Operand），操作数包含四种类型：常数（ImmediateNumber）、寄存器（Register）、偏移量（Offset）、标签（Label）。
![jupyter](./image/53.png)

以S0和S1相加，和存入S0为例，ADD S0, S1, S0代码的操作码设计如下：
![jupyter](./image/54.png)

机器语言的指令被虚拟机执行，最后产生输出。运行虚拟机的结果实例：

![jupyter](./image/55.png)
![jupyter](./image/56.png)


## 7 实验环境
本次设计在Visual Studio Code 1.67.2 IDE中编写JavaScript代码，Node.js版本为16.13.0，npm 8.5.5，使用Mocha Test Explorer 2.13.5扩展模块进行代码测试，引入第三方文件linkedlist、fs、chai。
![jupyter](./image/57.png)


## 8问题与解决
### 8.1 node_modules的创建
Node_modules文件在刚开始实验时并未成功创建，导致不能使用linkedlist、chai、mocha等依赖。查阅资料后，用npm命令进行了全局安装，在C盘中创建了相关的node_modules文件，之后使用npm安装的依赖也在同一个全局的路径下。但是在test文件的代码中进行测试引入依赖时，需要使用依赖存放的绝对路径，代码编写非常冗长。

![jupyter](./image/58.png)

由于本次设计需要用Node.js的require方法来获取依赖，需要本地安装，也就是默认的方式。直接在Visual Studio Code IDE中打开终端，输入install npm命令进行本地安装，即可在项目文件夹中生成node_modules文件。之后在代码中若需要使用require方法来获取依赖，即可直接输入依赖的名称，并且在Visual Studio Code IDE下，若存在此依赖文件，则会出现相关代码提示。

![jupyter](./image/59.png)


### 8.2 内存分配失败
在进行词法分析器测试时，Mocha模块出现了如下图的报错，提示如下：
![jupyter](./image/60.png)

通过查询报错语句的相关信息和increase-memory-limit依赖文件官方给出的README文档，在package.json文件中编写并Debug了script部分，配置文件和运行效果如下：
![jupyter](./image/61.png)
![jupyter](./image/62.png)


终端输出显示内存已经成功开辟，但是在测试时仍然存在问题。之后，还尝试了在环境变量中直接设置NODE_OPTIONS变量固定空间大小的最大值。

![jupyter](./image/63.png)

通过不同的方法均可以成功开辟新的空间，但是都不能正常进行测试。最后在检查代码时发现，Lexer.js文件中涉及到闭包的函数在书写时陷入了死循环，消耗了内存，修改代码后测试成功。所有测试用例正常运行，通过测试如下图所示：
![jupyter](./image/64.png)


## 9 总结感悟
本次设计基于JavaScript语言通过Visual Studio Code IDE设计了一套英文编程语言，WalsonScript。设计整体流程根据《编译原理》课程内容的讲解进行：词法分析、语法分析、语义分析、中间三地址代码表示、中间代码生成、语法制导翻译、代码优化。最终，WalsonScript在词法分析部分实现了对代码的分词断句，能判断词性，能处理var、int、float、string、boolean五种数据类型和”+\-*/><=!&|^%”几种基本运算符；在语法分析部分能处理以下几种语义：表达式语义Expression、单值成分语义Factor（变量语义Variable、数值语义Scalar）、声明语义Stmt（赋值语义AssignState、代码块语义Block、声明语义DeclareState、循环语义ForState、条件分支语义IfState、函数声明语义FuncState、函数变量语义FuncArgs、函数调用语义CallExpr、返回语义ReturnState）。

作为一种脚本语言，WalsonScript的提出在满足常规运算需求、编写基本代码的同时，比JavaScript书写更便捷，语法简单，功能简洁，句末无需追加分号；相比Python语言，无需考虑缩进和代码内部空格、换行的格式问题。明确定义了五种数据类型，在进行代码编写时避免了Python语言会遇到的接收数据类型不清晰、难以分类处理的问题。在保留字的定义上兼容性更强，比如对Boolean类型变量的定义，true/false大小写字母均合法。此外，WalsonScript使用流的数据结构获取、处理和存储输入的数据；规定了数字类型变量的合法区间和变量名的合法长度；使用Enum类存储节点类型，匹配处理时比较对象的地址。这些设计优化了WalsonScript进行计算时的效率，能满足基本运算的效率需求。WalsonScript在词法分析和翻译时，还针对不同的异常情况和代码非法编写设置了报错提示。最后编写虚拟机，执行了WalsonScript的代码。